In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5581,2020-11-15T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1476,217,1693,27114,...,1001,1422,11829,896,25806.0,15726.0,41532,816609,565253.0,NaN
5582,2020-11-15T17:00:00,ITA,9,Toscana,43.769231,11.255889,1734,274,2008,50300,...,1247,2653,25216,1879,63548.0,15855.0,79403,1331924,856660.0,NaN
5583,2020-11-15T17:00:00,ITA,10,Umbria,43.106758,12.388247,366,71,437,10658,...,319,657,7528,253,5315.0,13561.0,18876,360719,203439.0,Si fa presente come nel computo dei ricoverati...
5584,2020-11-15T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,148,12,160,2135,...,-4,143,2703,248,4641.0,605.0,5246,50230,29683.0,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5581,2020-11-15T17:00:00,19,Sicilia,1476,217,1693,27114,28807,1001,1422,11829,896,25806.0,15726.0,41532,816609,565253.0
5582,2020-11-15T17:00:00,9,Toscana,1734,274,2008,50300,52308,1247,2653,25216,1879,63548.0,15855.0,79403,1331924,856660.0
5583,2020-11-15T17:00:00,10,Umbria,366,71,437,10658,11095,319,657,7528,253,5315.0,13561.0,18876,360719,203439.0
5584,2020-11-15T17:00:00,2,Valle d'Aosta,148,12,160,2135,2295,-4,143,2703,248,4641.0,605.0,5246,50230,29683.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-11-15', '2020-11-14')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-15T17:00:00,1,Piemonte,4921,372,5293,65936,71229,2014,3682,49190,5046,52536.0,72929.0,125465,1273021,806208.0
1,2020-11-15T17:00:00,2,Valle d'Aosta,148,12,160,2135,2295,-4,143,2703,248,4641.0,605.0,5246,50230,29683.0
2,2020-11-15T17:00:00,3,Lombardia,7781,837,8618,153923,162541,6262,8060,138872,19367,241894.0,78886.0,320780,3550494,2152870.0
3,2020-11-15T17:00:00,5,Veneto,1934,241,2175,59331,61506,1801,2792,36525,2845,27442.0,73434.0,100876,2562531,989643.0
4,2020-11-15T17:00:00,6,Friuli Venezia Giulia,401,43,444,9483,9927,398,608,8482,528,16278.0,2659.0,18937,620174,253332.0
5,2020-11-15T17:00:00,7,Liguria,1393,115,1508,14230,15738,404,822,25530,2109,30385.0,12992.0,43377,527347,265754.0
6,2020-11-15T17:00:00,8,Emilia-Romagna,2285,246,2531,50670,53201,2639,2822,30275,5044,57557.0,30963.0,88520,1858993,972719.0
7,2020-11-15T17:00:00,9,Toscana,1734,274,2008,50300,52308,1247,2653,25216,1879,63548.0,15855.0,79403,1331924,856660.0
8,2020-11-15T17:00:00,10,Umbria,366,71,437,10658,11095,319,657,7528,253,5315.0,13561.0,18876,360719,203439.0
9,2020-11-15T17:00:00,11,Marche,483,77,560,13489,14049,630,707,8156,1111,23200.0,116.0,23316,368894,216441.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-14T17:00:00,1,Piemonte,4902,360,5262,63953,69215,2561,4471,47567,5001,50978.0,70805.0,121783,1259309,798081.0
1,2020-11-14T17:00:00,2,Valle d'Aosta,146,13,159,2140,2299,-64,245,2564,240,4513.0,590.0,5103,49650,29478.0
2,2020-11-14T17:00:00,3,Lombardia,7621,817,8438,147841,156279,5010,8129,137255,19186,236852.0,75868.0,312720,3511792,2135880.0
3,2020-11-14T17:00:00,5,Veneto,1869,228,2097,57608,59705,2417,3578,35563,2816,27166.0,70918.0,98084,2547409,984607.0
4,2020-11-14T17:00:00,6,Friuli Venezia Giulia,378,45,423,9106,9529,378,558,8284,516,15816.0,2513.0,18329,615669,252095.0
5,2020-11-14T17:00:00,7,Liguria,1402,108,1510,13824,15334,449,1091,25133,2088,29720.0,12835.0,42555,522497,264030.0
6,2020-11-14T17:00:00,8,Emilia-Romagna,2243,231,2474,48088,50562,2406,2637,30129,5008,56229.0,29470.0,85699,1846431,965000.0
7,2020-11-14T17:00:00,9,Toscana,1679,266,1945,49116,51061,932,2420,23854,1835,61678.0,15072.0,76750,1313187,848164.0
8,2020-11-14T17:00:00,10,Umbria,361,68,429,10347,10776,386,579,7201,242,5077.0,13142.0,18219,356390,201299.0
9,2020-11-14T17:00:00,11,Marche,484,74,558,12861,13419,585,779,8088,1102,22500.0,109.0,22609,365075,214152.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-15T17:00:00,1,Piemonte,4921,372,5293,65936,71229,2014,3682,49190,5046,52536.0,72929.0,125465,1273021,806208.0
1,2020-11-15T17:00:00,2,Valle d'Aosta,148,12,160,2135,2295,-4,143,2703,248,4641.0,605.0,5246,50230,29683.0
2,2020-11-15T17:00:00,3,Lombardia,7781,837,8618,153923,162541,6262,8060,138872,19367,241894.0,78886.0,320780,3550494,2152870.0
3,2020-11-15T17:00:00,4,Trentino-Alto Adige,849,76,925,12327,13252,359,896,15696,922,24011.0,5859.0,29870,605613,NaN
4,2020-11-15T17:00:00,5,Veneto,1934,241,2175,59331,61506,1801,2792,36525,2845,27442.0,73434.0,100876,2562531,989643.0
5,2020-11-15T17:00:00,6,Friuli Venezia Giulia,401,43,444,9483,9927,398,608,8482,528,16278.0,2659.0,18937,620174,253332.0
6,2020-11-15T17:00:00,7,Liguria,1393,115,1508,14230,15738,404,822,25530,2109,30385.0,12992.0,43377,527347,265754.0
7,2020-11-15T17:00:00,8,Emilia-Romagna,2285,246,2531,50670,53201,2639,2822,30275,5044,57557.0,30963.0,88520,1858993,972719.0
8,2020-11-15T17:00:00,9,Toscana,1734,274,2008,50300,52308,1247,2653,25216,1879,63548.0,15855.0,79403,1331924,856660.0
9,2020-11-15T17:00:00,10,Umbria,366,71,437,10658,11095,319,657,7528,253,5315.0,13561.0,18876,360719,203439.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-14T17:00:00,1,Piemonte,4902,360,5262,63953,69215,2561,4471,47567,5001,50978.0,70805.0,121783,1259309,798081.0
1,1,2020-11-14T17:00:00,2,Valle d'Aosta,146,13,159,2140,2299,-64,245,2564,240,4513.0,590.0,5103,49650,29478.0
2,2,2020-11-14T17:00:00,3,Lombardia,7621,817,8438,147841,156279,5010,8129,137255,19186,236852.0,75868.0,312720,3511792,2135880.0
3,19,2020-11-14T17:00:00,4,Trentino-Alto Adige,811,75,886,12007,12893,259,896,15186,895,23280.0,5694.0,28974,600205,NaN
4,3,2020-11-14T17:00:00,5,Veneto,1869,228,2097,57608,59705,2417,3578,35563,2816,27166.0,70918.0,98084,2547409,984607.0
5,4,2020-11-14T17:00:00,6,Friuli Venezia Giulia,378,45,423,9106,9529,378,558,8284,516,15816.0,2513.0,18329,615669,252095.0
6,5,2020-11-14T17:00:00,7,Liguria,1402,108,1510,13824,15334,449,1091,25133,2088,29720.0,12835.0,42555,522497,264030.0
7,6,2020-11-14T17:00:00,8,Emilia-Romagna,2243,231,2474,48088,50562,2406,2637,30129,5008,56229.0,29470.0,85699,1846431,965000.0
8,7,2020-11-14T17:00:00,9,Toscana,1679,266,1945,49116,51061,932,2420,23854,1835,61678.0,15072.0,76750,1313187,848164.0
9,8,2020-11-14T17:00:00,10,Umbria,361,68,429,10347,10776,386,579,7201,242,5077.0,13142.0,18219,356390,201299.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-15T17:00:00,1,Piemonte,4921,372,5293,65936,71229,2014,3682,49190,5046,52536.0,72929.0,125465,1273021,806208.0,4341375
1,2020-11-15T17:00:00,2,Valle d'Aosta,148,12,160,2135,2295,-4,143,2703,248,4641.0,605.0,5246,50230,29683.0,125501
2,2020-11-15T17:00:00,3,Lombardia,7781,837,8618,153923,162541,6262,8060,138872,19367,241894.0,78886.0,320780,3550494,2152870.0,10103969
3,2020-11-15T17:00:00,4,Trentino-Alto Adige,849,76,925,12327,13252,359,896,15696,922,24011.0,5859.0,29870,605613,NaN,1074819
4,2020-11-15T17:00:00,5,Veneto,1934,241,2175,59331,61506,1801,2792,36525,2845,27442.0,73434.0,100876,2562531,989643.0,4907704
5,2020-11-15T17:00:00,6,Friuli Venezia Giulia,401,43,444,9483,9927,398,608,8482,528,16278.0,2659.0,18937,620174,253332.0,1211357
6,2020-11-15T17:00:00,7,Liguria,1393,115,1508,14230,15738,404,822,25530,2109,30385.0,12992.0,43377,527347,265754.0,1543127
7,2020-11-15T17:00:00,8,Emilia-Romagna,2285,246,2531,50670,53201,2639,2822,30275,5044,57557.0,30963.0,88520,1858993,972719.0,4467118
8,2020-11-15T17:00:00,9,Toscana,1734,274,2008,50300,52308,1247,2653,25216,1879,63548.0,15855.0,79403,1331924,856660.0,3722729
9,2020-11-15T17:00:00,10,Umbria,366,71,437,10658,11095,319,657,7528,253,5315.0,13561.0,18876,360719,203439.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-15T17:00:00,1,Piemonte,4921,372,5293,65936,71229,2014,3682,49190,5046,52536.0,72929.0,125465,1273021,806208.0,4341375,575
1,2020-11-15T17:00:00,2,Valle d'Aosta,148,12,160,2135,2295,-4,143,2703,248,4641.0,605.0,5246,50230,29683.0,125501,20
2,2020-11-15T17:00:00,3,Lombardia,7781,837,8618,153923,162541,6262,8060,138872,19367,241894.0,78886.0,320780,3550494,2152870.0,10103969,1036
3,2020-11-15T17:00:00,4,Trentino-Alto Adige,849,76,925,12327,13252,359,896,15696,922,24011.0,5859.0,29870,605613,NaN,1074819,106
4,2020-11-15T17:00:00,5,Veneto,1934,241,2175,59331,61506,1801,2792,36525,2845,27442.0,73434.0,100876,2562531,989643.0,4907704,1016
5,2020-11-15T17:00:00,6,Friuli Venezia Giulia,401,43,444,9483,9927,398,608,8482,528,16278.0,2659.0,18937,620174,253332.0,1211357,180
6,2020-11-15T17:00:00,7,Liguria,1393,115,1508,14230,15738,404,822,25530,2109,30385.0,12992.0,43377,527347,265754.0,1543127,209
7,2020-11-15T17:00:00,8,Emilia-Romagna,2285,246,2531,50670,53201,2639,2822,30275,5044,57557.0,30963.0,88520,1858993,972719.0,4467118,634
8,2020-11-15T17:00:00,9,Toscana,1734,274,2008,50300,52308,1247,2653,25216,1879,63548.0,15855.0,79403,1331924,856660.0,3722729,523
9,2020-11-15T17:00:00,10,Umbria,366,71,437,10658,11095,319,657,7528,253,5315.0,13561.0,18876,360719,203439.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-15T17:00:00,3,Lombardia,7781,837,8618,153923,162541,6262,8060,...,80.79,6082,-69,38702,16990.0,181,1617,20.83,0.07977,3.17
1,2020-11-15T17:00:00,15,Campania,2224,194,2418,85839,88257,3011,3771,...,38.42,2857,420,24948,19523.0,35,725,15.12,0.06518,1.92
2,2020-11-15T17:00:00,1,Piemonte,4921,372,5293,65936,71229,2014,3682,...,64.70,1983,-789,13712,8127.0,45,1623,26.85,0.08481,2.89
3,2020-11-15T17:00:00,8,Emilia-Romagna,2285,246,2531,50670,53201,2639,2822,...,38.80,2582,185,12562,7719.0,36,146,22.46,0.06317,1.98
4,2020-11-15T17:00:00,5,Veneto,1934,241,2175,59331,61506,1801,2792,...,23.72,1723,-786,15122,5036.0,29,962,18.46,0.05689,2.06
5,2020-11-15T17:00:00,9,Toscana,1734,274,2008,50300,52308,1247,2653,...,52.39,1184,233,18737,8496.0,44,1362,14.16,0.07126,2.13
6,2020-11-15T17:00:00,12,Lazio,3159,274,3433,63876,67309,2351,2612,...,32.35,2267,-385,22434,18750.0,21,240,11.64,0.04453,1.44
7,2020-11-15T17:00:00,19,Sicilia,1476,217,1693,27114,28807,1001,1422,...,36.90,985,-307,7416,4517.0,36,385,19.17,0.02862,0.84
8,2020-11-15T17:00:00,16,Puglia,1261,180,1441,23261,24702,702,905,...,48.78,728,-836,6070,3292.0,12,191,14.91,0.02258,0.86
9,2020-11-15T17:00:00,4,Trentino-Alto Adige,849,76,925,12327,13252,359,896,...,71.70,320,0,5408,NaN,27,510,16.57,0.08336,2.78


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Lombardia,837,199,1036,80.79
1,Trentino-Alto Adige,76,30,106,71.70
2,Piemonte,372,203,575,64.70
3,Umbria,71,46,117,60.68
4,Valle d'Aosta,12,8,20,60.00
5,Liguria,115,94,209,55.02
6,Marche,77,66,143,53.85
7,Toscana,274,249,523,52.39
8,Puglia,180,189,369,48.78
9,Abruzzo,59,89,148,39.86


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['nuovi_positivi'] / (df['totale_positivi'] - df['nuovi_positivi'] + df['nuovi_dimessi_guariti'] + df['nuovi_deceduti'])).replace(np.inf, 0) * 100).round(3)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv
